In [1]:
import os
from concurrent.futures import ProcessPoolExecutor
import shutil
import numpy as np
import pandas as pd

In [2]:
enable_prallel_processing = 0 # 0-disable 1-enable
no_sub_to_run_in_parallel = 4
sub_det_df = pd.read_excel('data/rs_fmri_sub_details_WO_AS.xlsx')
cur_sub_list_before = sub_det_df['Subject_ID_before'].to_numpy()
cur_sub_list_after = sub_det_df['Subject_ID_after'].to_numpy()
subject_ids = np.concatenate((cur_sub_list_before,cur_sub_list_after))
print('# of subjects to Process:', len(subject_ids))

# of subjects to Process: 178


In [4]:
data_dir = '/data-braingut/bharath/fmri_bandpass'
if not (os.path.isdir('/data-braingut/bharath/fmri_pre_process_native_space/BP_3PC_anaticor_motreg_noscale_4blur')):
    os.mkdir('/data-braingut/bharath/fmri_pre_process_native_space/BP_3PC_anaticor_motreg_noscale_4blur')    

In [5]:
def run_afni_proc_Native_space_partial(sub_id, data_dir):
    cmdline = ('afni_proc.py -subj_id %s \
         -blocks mask regress \
         -dsets %s/%s/pb04.%s.r01.blur+orig \
         -mask_import FSvent %s/%s/follow_ROI_FSvent+orig \
         -mask_import FSWe %s/%s/follow_ROI_FSWe+orig \
         -regress_bandpass 0.01 0.1 \
         -regress_motion_file %s/%s/dfile_rall.1D \
         -regress_motion_per_run \
         -regress_ROI_PC FSvent 3 \
         -regress_ROI_PC_per_run FSvent \
         -regress_anaticor_fast \
         -regress_anaticor_label FSWe \
         -regress_apply_mot_types demean deriv \
         -out_dir /data-braingut/bharath/fmri_pre_process_native_space/BP_3PC_anaticor_motreg_noscale_4blur/%s \
         -html_review_style basic')%(sub_id,data_dir,sub_id,sub_id,data_dir,sub_id,data_dir,sub_id,data_dir,sub_id,sub_id)
    #     -regress_censor_extern %s/%s/outcount_%s_censor.1D\    
    os.system(cmdline)
#     print(cmdline)
    cmdline1 = ('tcsh -xef proc.%s 2>&1 | tee output.proc.%s')%(sub_id,sub_id)
    os.system(cmdline1)
#     print(cmdline1)

In [ ]:
if(enable_prallel_processing == 1):
    with ProcessPoolExecutor(max_workers = no_sub_to_run_in_parallel) as executor:
        for i in subject_ids:
            executor.submit(run_afni_proc_Native_space_partial, i, data_dir)  
else:
    for i in subject_ids:
        run_afni_proc_Native_space_partial(i, data_dir)  

In [5]:
moveto_dir = '/data-braingut/bharath/fmri_pre_process_native_space/BP_3PC_anaticor_motreg_noscale_4blur/output_proc'
if not (os.path.isdir(moveto_dir)):
    os.mkdir('moveto_dir')
for sub_id in subject_ids:
    shutil.move('output.proc.%s'%sub_id, moveto_dir)
    shutil.move('proc.%s'%sub_id, moveto_dir)  